In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

from utils import tf_config

dataset="01"
memory_limit=1500
interval_eeg=10
time_length=10#how big is the sequence to be predicted

tf_config.set_seed(seed=2)#02 [2, 3, 5, 7, 11]
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

import GPyOpt

import argparse

from utils import preprocess_data, search_algorithms, train, losses_utils, metrics, eeg_utils, data_utils

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

from pathlib import Path

import time

import pickle

n_individuals=getattr(data_utils, "n_individuals_"+dataset)

with tf.device('/CPU:0'):
    train_data, test_data = preprocess_data.dataset(dataset, n_individuals=n_individuals,
                                            interval_eeg=interval_eeg, 
                                            ind_volume_fit=False,
                                            standardize_fmri=True,
                                            time_length=time_length,
                                            iqr=False,
                                            verbose=True)
    eeg_train, fmri_train =train_data
    eeg_test, fmri_test = test_data

I: Starting to Load Data
I: Finished Loading Data
I: Pairs Created


In [2]:
eeg_train.shape, fmri_train.shape


((224, 64, 134, 20, 1), (224, 64, 64, 30, 10))

In [3]:
eeg_test.shape, fmri_test.shape

((56, 64, 134, 20, 1), (56, 64, 64, 30, 10))

## Build model

In [4]:
theta = (0.002980911194116198, 0.0004396489214334123, (9, 9, 4), (1, 1, 1), 4, (7, 7, 7), 4, True, True, True, True, 3, 1)
#unroll hyperparameters
learning_rate=float(theta[0])
weight_decay = float(theta[1])
kernel_size = theta[2]
stride_size = theta[3]
batch_size=int(theta[4])
latent_dimension=theta[5]
n_channels=int(theta[6])
max_pool=bool(theta[7])
batch_norm=bool(theta[8])
skip_connections=bool(theta[9])
dropout=bool(theta[10])
n_stacks=int(theta[11])
outfilter=int(theta[12])
local=True
with open(str(Path.home())+"/eeg_to_fmri/na_models_eeg/na_specification_2", "rb") as f:
    na_specification_eeg = pickle.load(f)
with open(str(Path.home())+"/eeg_to_fmri/na_models_fmri/na_specification_2", "rb") as f:
    na_specification_fmri = pickle.load(f)
fmri_args=(latent_dimension, fmri_train.shape[1:], kernel_size, stride_size, n_channels, max_pool, batch_norm, weight_decay, skip_connections,n_stacks, True, False, outfilter, dropout, None, False, na_specification_fmri)

In [5]:
import importlib
importlib.reload(eeg_to_fmri)

with tf.device('/CPU:0'):
    model = eeg_to_fmri.EEG_to_fMRI(latent_dimension, eeg_train.shape[1:], na_specification_eeg, n_channels,
                    weight_decay=weight_decay, skip_connections=True, batch_norm=True, fourier_features=True,
                    random_fourier=True, topographical_attention=False, conditional_attention_style=False,
                    conditional_attention_style_prior=False, local=True, seed=None, time_length=time_length,
                    fmri_args=fmri_args)
    
    model.build(eeg_train.shape, fmri_train.shape)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = losses_utils.TemporalLoss()#we will see about the loss, if temporal or spatial

    train_set = tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(batch_size)
    test_set= tf.data.Dataset.from_tensor_slices((eeg_test, fmri_test)).batch(1)

2023-07-04 16:19:15.236399: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1101004800 exceeds 10% of free system memory.


## Train the model

In [6]:
train.train(train_set, model, optimizer, loss_fn, epochs=1, u_architecture=True, val_set=None, verbose=True, verbose_batch=True)

2023-07-04 16:19:16.074031: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1101004800 exceeds 10% of free system memory.
2023-07-04 16:19:18.727393: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-07-04 16:19:18.728063: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


2023-07-04 16:20:07.259204: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 17.79MiB (rounded to 18653440)requested by op Sub
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-04 16:20:07.264575: W tensorflow/core/common_runtime/bfc_allocator.cc:491] ****************xxxx********************************************************************************
2023-07-04 16:20:07.270596: W tensorflow/core/framework/op_kernel.cc:1733] RESOURCE_EXHAUSTED: failed to allocate memory


ResourceExhaustedError: failed to allocate memory [Op:Sub]